In [ ]:
import pandas as pd

# Dataframe
df = pd.read_csv("sleep_health.csv")

# NRS is Poor subjective sleep quality -> values: (1 = Present, 2 = Absent)
df_2015 = pd.DataFrame({
    "Poor_Sleep": df["2015NRS"],
    "Gender": df["gender"], # (1 = male, 2 = female)
    "Age": df["2015Age"],
    "BMI": df["2015BMI"],
    "Smoking": df["2015Smoking"], 
    "Exercise": df["2015EX"],
    "Diabetes": df["2015Diabetes"],
    "Drinking_Habit": df["2015DrinkHabit"] # (1 = Everyday, 2 = Sometimes, 3 = Rarely, Does not)
})

print(df_2015.head())

   Poor_Sleep  Gender  Age   BMI  Smoking  Exercise  Diabetes  Drinking_Habit
0           1       1   65  26.0        0         1         1               1
1           1       1   73  20.8        0         2         0               1
2           1       1   64  23.5        1         2         0               3
3           2       1   67  25.0        0         2         0               3
4           1       1   73  25.2        0         1         0               2
